# Pre-Class 4.1

In [1]:
import matplotlib
import pylab
import random
import numpy as np

In [96]:
class TrafficSimulator(object):
    
    def __init__(self, length=50, density=0.2, vMax=5, slowProb=0.5):
        '''
        Initialize new TrafficSimulator objects with the default parameter
        settings.

        Inputs:

         * length (int): The number of cells in the cellular automaton.
           Default: 50.

         * density (float): The proportion of cells that will be active (contain
           a car) when the cellular automaton is initialized. The active cells
           will be at random locations along the length of the CA. Default: 0.2.

         * vMax (int): The maximum velocity of the cars. Default: 5.

         * slowProb (float): The probability of a car to randomly slow down.
           Default: 0.5.
        '''
        self.length = length
        self.density = density
        self.vMax = vMax
        self.slowProb = slowProb

        # Arrays for storing the current state and the next state (when we are
        # busy doing a state update).
        self.current_state = np.ones(self.length) * -1
        self.next_state = np.ones(self.length) * -1

    def initialize(self):
        '''
        This will be called by the interactive pycxsimulator whenever the Reset
        button is clicked.
        '''
        # Set up a random initial state where the fraction of 0s in the array
        # equals the density parameter.
        random_indices = np.random.choice(
            range(self.length),
            size=int(round(self.density * self.length)),
            replace=False)
        self.current_state.fill(-1)
        self.current_state[random_indices] = 0
        
        # Set the velocities to the max possible (i.e, distant - 1 or max velocity)
        for i in range(self.length):
            if self.current_state[i] != -1:
                for j in range(1, self.vMax+1):
                    if self.current_state[(i+j)%self.length] != -1:
                        dist = j
                        break
                    dist = self.vMax+1
                self.current_state[i] = dist-1

        self.time = 0
        
    def draw(self):
        '''
        Draw the current state of the cellular automaton.
        '''
#         if self.interactive:
#             pylab.cla()  # Clear the current figure
#         else:
#             pylab.figure()  # Create a new figure
#         pylab.pcolor(
#             self.current_state.reshape((1, self.length)),
#             vmin = 0,
#             vmax = 1,
#             cmap = pylab.cm.binary)
#         pylab.axis('image')
#         pylab.title('density = ' + str(self.density) + '; t = ' + str(self.time))
#         if not self.interactive:
#             pylab.show()
        print(''.join('.' if x == -1 else str(int(x)) for x in self.current_state[:70]))

    def update_velocities(self):
        '''
        Update all velocities according to rules 1-3
        '''
        for i in range(self.length):
            if self.current_state[i] != -1:
                # Calculate the distance to next car (if distance is long, set as vMax+1)
                for j in range(1, self.vMax+1):
                    if self.current_state[(i+j)%self.length] != -1:
                        dist = j
                        break
                    dist = self.vMax+1
                # Rule 1: acceleration
                if (self.current_state[i] < self.vMax) and (dist > self.current_state[i]+1):
                    self.current_state[i] += 1
                # Rule 2: slowing down
                if self.current_state[i] >= dist:
                    self.current_state[i] = dist-1
                # Rule 3: randomization
                if self.current_state[i] != 0:
                    if random.random() < self.slowProb:
                        self.current_state[i] -= 1
        
    def step(self):
        '''
        Update the positions based on updated velocities
        '''
        # Update the new positions (rule 4)
        self.next_state = np.ones(self.length) * -1
        for i in range(self.length):
            if self.current_state[i] != -1:
                self.next_state[int((i + self.current_state[i])%self.length)] = self.current_state[i]
        
        # Swap next state and current state
        self.current_state, self.next_state = self.next_state, self.current_state
        self.time += 1



## Nagel & Schreckenberg - figure 1

In [98]:
sim = TrafficSimulator(length=100, density=0.03, vMax=5, slowProb=0.5)
sim.initialize()
sim.current_state
# sim.draw()
for i in range(20):
    sim.update_velocities()
    sim.draw()
    sim.step()

....5.................................5...............4...............
.........5.................................4..............5...........
..............4................................4...............5......
..................5................................4................5.
.......................5...............................4..............
............................5..............................4..........
.................................4.............................5......
.....................................5..............................5.
..........................................5...........................
...............................................4......................
.5.................................................5..................
......4.................................................5.............
..........5..................................................4........
...............5.................................................5....
......

## Nagel & Schreckenberg - figure 2

In [109]:
sim = TrafficSimulator(length=100, density=0.1, vMax=5, slowProb=0.5)
sim.initialize()
sim.current_state
# sim.draw()
for i in range(30):
    sim.update_velocities()
    sim.draw()
    sim.step()

..4............5.....4.....05......................1.0.4......5.......
......5.............4....0.0.....5..................01.....5.......4..
...........5............00.0..........4.............1.2.........5.....
................5.......00.0..............5..........2..3............4
.....................2..00.1...................4.......3...4..........
....5..................001..2......................4......3....5......
.........4.............00.1...2........................4.....4......5.
.............5.........01..2....3..........................4.....5....
5.................3....1.1...2.....3...........................4......
.....4...............2..1.2....3......3............................5..
.5.......5.............1.2..3.....3......4............................
......5.......4.........1..3...3.....4.......5........................
...........4......4......1....2...3......5........5...................
5..............5......2...1.....3....4........5........4..............
.....5